# ENSO

This notebook provides an overview of running the ENSO metrics. More information can be found in the [README]( ). Example parameter files are located in the [PMP sample setups]( ). 

The ENSO metrics demo requires downloading a large sample data set and installing the CLIVAR ENSO Metrics package. The first sections of this notebook will walk you through these steps.

## Download demo data

The ENSO metric requires a different set of sample data than the rest of the PMP metrics. This section of the notebook will download that data to your chosen location and generate a basic parameter file.

In [11]:
# Lets get the file containing the data needed for this demo
import requests
r = requests.get("https://pcmdiweb.llnl.gov/pss/pmpdata/pmp_enso_tutorial_files.txt")
with open("enso_data_files.txt","wb") as f:
    f.write(r.content)

If you want to change the location where the demo data and output are stored, you can do so here:

In [12]:
# This is where you will be downloading the sample_data
demo_data_directory = "enso_data"
# this line is where your output will be stored
demo_output_directory = "enso_output"

Then download the data. The total sample data size is X. This may take several minutes.

In [13]:
# Let's download the files
import cdat_info
try:
    cdat_info.download_sample_data_files("enso_data_files.txt", demo_data_directory)
    print("All files downloaded")
except:
    print("Download failed")

MD5: enso_data_files.txt
All files downloaded


After downloading the data, we generate the parameter file for this demo.

In [14]:
from download_sample_data import generate_parameter_files
filenames=["basic_enso_param.py.in"]
generate_parameter_files(demo_data_directory, demo_output_directory, filenames=filenames)

Preparing parameter file: basic_enso_param.py
Saving User Choices


## Environment

The [ENSO Metrics package](https://github.com/CLIVAR-PRP/ENSO_metrics) must be installed to run the PMP ENSO driver. This section will clone the ENSO Metrics repository and *install ENSO Metrics in your current environment*. Set the `enso_install_location` below to chose where to clone the ENSO Metrics repository.

In [15]:
enso_install_location = "."

To clone and install the ENSO Metrics package, un-comment the next cell (delete the quotes on lines 1 and 6) and run it.

In [21]:
"""
%%bash -s "$enso_install_location"
cd $1
git clone https://github.com/CLIVAR-PRP/ENSO_metrics.git
cd ENSO_metrics
python setup.py install
"""

running install
running build
running build_py
creating build
creating build/lib
creating build/lib/EnsoMetrics
copying lib/EnsoPlotLib.py -> build/lib/EnsoMetrics
copying lib/version.py -> build/lib/EnsoMetrics
copying lib/EnsoMetricsLib.py -> build/lib/EnsoMetrics
copying lib/__init__.py -> build/lib/EnsoMetrics
copying lib/EnsoCollectionsLib.py -> build/lib/EnsoMetrics
copying lib/EnsoErrorsWarnings.py -> build/lib/EnsoMetrics
copying lib/KeyArgLib.py -> build/lib/EnsoMetrics
copying lib/EnsoUvcdatToolsLib.py -> build/lib/EnsoMetrics
copying lib/EnsoToolsLib.py -> build/lib/EnsoMetrics
copying lib/EnsoComputeMetricsLib.py -> build/lib/EnsoMetrics
creating build/lib/EnsoPlots
copying plots/EnsoPlotToolsLib.py -> build/lib/EnsoPlots
copying plots/EnsoMetricPlot.py -> build/lib/EnsoPlots
copying plots/__init__.py -> build/lib/EnsoPlots
copying plots/EnsoPlotTemplate.py -> build/lib/EnsoPlots
running build_scripts
creating build/scripts-3.8
copying scripts/driver_plot_portraitplot.py ->

Cloning into 'ENSO_metrics'...


## Usage

ENSO can be run on the command line or with using a parameter file

For help, type:  
```
enso_driver.py --help
```

In [17]:
%%bash
enso_driver.py --help

usage: enso_driver.py [-h] [--parameters PARAMETERS]
                      [--diags OTHER_PARAMETERS [OTHER_PARAMETERS ...]]
                      [--mip MIP] [--exp EXP] [--modpath MODPATH]
                      [--modpath_lf MODPATH_LF]
                      [--modnames MODNAMES [MODNAMES ...]] [-r REALIZATION]
                      [--reference_data_path REFERENCE_DATA_PATH]
                      [--reference_data_lf_path REFERENCE_DATA_LF_PATH]
                      [--metricsCollection METRICSCOLLECTION]
                      [--json_name JSON_NAME] [--netcdf_name NETCDF_NAME]
                      [--results_dir RESULTS_DIR] [--case_id CASE_ID]
                      [--obs_catalogue OBS_CATALOGUE]
                      [--obs_cmor_path OBS_CMOR_PATH] [-d [DEBUG]]
                      [--obs_cmor [OBS_CMOR]] [--nc_out [NC_OUT]]

optional arguments:
  -h, --help            show this help message and exit
  --parameters PARAMETERS, -p PARAMETERS
  --diags OTHER_PARAMETERS [OTHER_PA

### Basic example

The ENSO driver settings can be provided in a parameter file, which is shown here.

In [18]:
with open("basic_enso_param.py") as f:
    print(f.read())

import os

#
#  OPTIONS ARE SET BY USER IN THIS FILE AS INDICATED BELOW BY:
#
#

# MODELS
modnames = ['ACCESS1-0']
mip = 'cmip5'  # cmip5, cmip6
exp = 'historical'  # historical, piControl
realization = 'r1i1p1'
modpath = 'enso_data/CMIP5_demo_data/%(variable)_Amon_%(model)_historical_%(realization)_185001-200512.nc'
modpath_lf = 'enso_data/CMIP5_demo_data/sftlf_fx_%(model)_amip_r0i0p0.nc'

# OBSERVATIONS
obs_cmor = True
obs_cmor_path = "enso_data/PCMDIobs2"
obs_catalogue = "demo_data/pcmdiobs-CEM2020_monthly_bySource_catalogue_v20210422.json"
#reference_data_lf_path = {
#    'GPCPv2.3': 'enso_data/gpcp_25_lsmask.nc'
#}

# METRICS COLLECTION
metricsCollection = 'ENSO_perf'  # ENSO_perf, ENSO_tel, ENSO_proc

# OUTPUT
case_id = 'basicTestEnso'
results_dir = os.path.join('enso_output',case_id)

json_name = '%(mip)_%(exp)_%(metricsCollection)_%(case_id)_%(model)_%(realization)'
netcdf_name = json_name
nc_out = False



The next cell runs the ENSO driver using the basic parameter file. This may take several minutes.

In [19]:
%%bash
enso_driver.py -p basic_enso_param.py

mip: cmip5
exp: historical
models: ['ACCESS1-0']
realization:  r1i1p1
mc_name: ENSO_perf
outdir: enso_output/basicTestEnso
netcdf_path: enso_output/basicTestEnso
debug: False
obs_cmor: True
obs_cmor_path: enso_data/PCMDIobs2
egg_pth: /Users/ordonez4/Library/Caches/Python-Eggs/pcmdi_metrics-v1.2.1_690_g281220b-py3.8.egg-tmp/share/pmp
output directory for graphics:enso_output/basicTestEnso
output directory for diagnostic_results:enso_output/basicTestEnso
output directory for metrics_results:enso_output/basicTestEnso
list_variables: ['pr', 'sst', 'taux']
list_obs: ['20CR', 'AVISO-1-0', 'CERES-EBAF-4-1', 'CMAP-V1902', 'ERA-20C', 'ERA-5', 'ERA-INT', 'GPCP-2-3', 'HadISST-1-1', 'REMSS-PRW-v07r01', 'TropFlux-1-0']
PMPdriver: dict_obs readin end
models: ['ACCESS1-0']
 ----- model:  ACCESS1-0  ---------------------
PMPdriver: var loop start for model  ACCESS1-0
realization: r1i1p1
 --- run:  r1i1p1  ---
 --- var:  pr  ---
var_in_file: pr
var, areacell_in_file, realm: pr areacella atmos
path:  en

/Users/ordonez4/miniconda3/envs/test_pcmdi/lib/python3.8/site-packages/cdms2/MV2.py:318: Warning: arguments order for compress function has changed
it is now: MV2.copmress(array,condition), if your code seems to not react or act wrong to a call to compress, please check this
  warnings.warn(
INFO::2021-05-28 14:53::pcmdi_metrics:: Results saved to a json file: /Users/ordonez4/Documents/git/pcmdi_metrics/doc/jupyter/Demo/enso_output/basicTestEnso/cmip5_historical_ENSO_perf_basicTestEnso_ACCESS1-0_r1i1p1.json
INFO::2021-05-28 14:53::pcmdi_metrics:: Results saved to a json file: /Users/ordonez4/Documents/git/pcmdi_metrics/doc/jupyter/Demo/enso_output/basicTestEnso/cmip5_historical_ENSO_perf_basicTestEnso_ACCESS1-0_r1i1p1_diveDown.json


This run saved metrics to two files:  
basicTestEnso/cmip5_historical_ENSO_perf_basicTestEnso_ACCESS1-0_r1i1p1.json   basicTestEnso/cmip5_historical_ENSO_perf_basicTestEnso_ACCESS1-0_r1i1p1_diveDown.json

The results are quite lengthy, but we will show a section of them here for the ENSO Ampl metric.

In [20]:
import json
metrics_file=demo_output_directory+"/basicTestEnso/cmip5_historical_ENSO_perf_basicTestEnso_ACCESS1-0_r1i1p1.json"
with open(metrics_file) as f:
    results = json.load(f)["RESULTS"]["model"]["ACCESS1-0"]["r1i1p1"]["value"]["EnsoAmpl"]
print(json.dumps(results, indent = 2))

{
  "diagnostic": {
    "20CR": {
      "value": 0.7407960057502115,
      "value_error": 0.062166219832622445
    },
    "ACCESS1-0_r1i1p1": {
      "value": 0.6630260736952586,
      "value_error": 0.0530845705527287
    },
    "ERA-20C": {
      "value": 0.8265126323027572,
      "value_error": 0.07844910735406072
    },
    "ERA-5": {
      "value": 0.9075909980564855,
      "value_error": 0.14350273688619733
    },
    "ERA-INT": {
      "value": 0.9001341048707652,
      "value_error": 0.1423236985494241
    }
  },
  "metric": {
    "20CR": {
      "value": 10.498157583367986,
      "value_error": 14.67671016308911
    },
    "ERA-20C": {
      "value": 19.780285529575835,
      "value_error": 14.03683572911868
    },
    "ERA-5": {
      "value": 26.946600934224556,
      "value_error": 17.39970955404781
    },
    "ERA-INT": {
      "value": 26.341411784363938,
      "value_error": 17.543852271121864
    }
  }
}


### NetCDF metrics

To save individual metrics in netCDF format, use the `--nc_out` flag.

In [22]:
%%bash
enso_driver.py -p basic_enso_param.py --nc_out True --case_id ncResults

mip: cmip5
exp: historical
models: ['ACCESS1-0']
realization:  r1i1p1
mc_name: ENSO_perf
outdir: enso_output/basicTestEnso
netcdf_path: enso_output/basicTestEnso
debug: False
obs_cmor: True
obs_cmor_path: enso_data/PCMDIobs2
egg_pth: /Users/ordonez4/Library/Caches/Python-Eggs/pcmdi_metrics-v1.2.1_690_g281220b-py3.8.egg-tmp/share/pmp
output directory for graphics:enso_output/basicTestEnso
output directory for diagnostic_results:enso_output/basicTestEnso
output directory for metrics_results:enso_output/basicTestEnso
list_variables: ['pr', 'sst', 'taux']
list_obs: ['20CR', 'AVISO-1-0', 'CERES-EBAF-4-1', 'CMAP-V1902', 'ERA-20C', 'ERA-5', 'ERA-INT', 'GPCP-2-3', 'HadISST-1-1', 'REMSS-PRW-v07r01', 'TropFlux-1-0']
PMPdriver: dict_obs readin end
models: ['ACCESS1-0']
 ----- model:  ACCESS1-0  ---------------------
PMPdriver: var loop start for model  ACCESS1-0
realization: r1i1p1
 --- run:  r1i1p1  ---
 --- var:  pr  ---
var_in_file: pr
var, areacell_in_file, realm: pr areacella atmos
path:  en

/Users/ordonez4/miniconda3/envs/test_pcmdi/lib/python3.8/site-packages/cdms2/MV2.py:318: Warning: arguments order for compress function has changed
it is now: MV2.copmress(array,condition), if your code seems to not react or act wrong to a call to compress, please check this
  warnings.warn(
INFO::2021-05-28 15:27::pcmdi_metrics:: Results saved to a json file: /Users/ordonez4/Documents/git/pcmdi_metrics/doc/jupyter/Demo/enso_output/basicTestEnso/cmip5_historical_ENSO_perf_basicTestEnso_ACCESS1-0_r1i1p1.json
INFO::2021-05-28 15:28::pcmdi_metrics:: Results saved to a json file: /Users/ordonez4/Documents/git/pcmdi_metrics/doc/jupyter/Demo/enso_output/basicTestEnso/cmip5_historical_ENSO_perf_basicTestEnso_ACCESS1-0_r1i1p1_diveDown.json


In [24]:
!ls {demo_output_directory + "/ncResults/*.nc"}

enso_output/ncResults/cmip5_historical_ENSO_perf_basicTestEnso_ACCESS1-0_r1i1p1_BiasPrLatRmse.nc
enso_output/ncResults/cmip5_historical_ENSO_perf_basicTestEnso_ACCESS1-0_r1i1p1_BiasPrLonRmse.nc
enso_output/ncResults/cmip5_historical_ENSO_perf_basicTestEnso_ACCESS1-0_r1i1p1_BiasSstLonRmse.nc
enso_output/ncResults/cmip5_historical_ENSO_perf_basicTestEnso_ACCESS1-0_r1i1p1_BiasTauxLonRmse.nc
enso_output/ncResults/cmip5_historical_ENSO_perf_basicTestEnso_ACCESS1-0_r1i1p1_EnsoAmpl.nc
enso_output/ncResults/cmip5_historical_ENSO_perf_basicTestEnso_ACCESS1-0_r1i1p1_EnsoDuration.nc
enso_output/ncResults/cmip5_historical_ENSO_perf_basicTestEnso_ACCESS1-0_r1i1p1_EnsoSeasonality.nc
enso_output/ncResults/cmip5_historical_ENSO_perf_basicTestEnso_ACCESS1-0_r1i1p1_EnsoSstDiversity_2.nc
enso_output/ncResults/cmip5_historical_ENSO_perf_basicTestEnso_ACCESS1-0_r1i1p1_EnsoSstLonRmse.nc
enso_output/ncResults/cmip5_historical_ENSO_perf_basicTestEnso_ACCESS1-0_r1i1p1_EnsoSstSkew.nc
enso_output/ncResults/cmip5